# Scales of thirds

All valid scales have either a minor or a major third interval between every other scale not. Only scales staisfying this requirement are modes of major, melodic minor, harmonic minor, harmonic major, octatonic, augmented, and hole tone scales.

Given a set of pitch classes:

- in how many ways these can be a subset of any scales
- list allowed transformations of this (i.e. move a note up or down a step)

## TODO:

I wrote a harmonicity calculator and most consonant/dissonant basse voice finder. Where is it?

In [ ]:
from collections import Counter, defaultdict, deque
from copy import copy

In [ ]:
def contains(this, other):
    return all([(a and b) or not a for a, b in zip(other, this)])

In [ ]:
class Necklace(deque):
    def __init__(self, value):
        super().__init__(value)
        # number of transpositions
        n_t = 0
        me, you = deque(value), deque(value)
        for idx in range(len(value)):
            if me == you:
                n_t += 1
            you.rotate()
        self.n_t = 12 // n_t

    def __eq__(self, other):
        assert len(self) == len(other)
        other = copy(other)
        for idx in range(self.n_t):
            if deque.__eq__(self, other):
                return True
            other.rotate()
        return False

    def __ge__(self, other):
        assert len(self) == len(other)
        other = copy(other)
        result = 0
        for idx in range(self.n_t):
            if contains(self, other):
                result += 1
            other.rotate()
        return result

    def __hash__(self):
        cache = [self.copy() for _ in range(len(self))]
        for n, val in enumerate(cache):
            val.rotate(n)
            cache[n] = tuple(val)

        return hash(frozenset(Counter(tuple(cache)).items()))

In [ ]:
major = [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1]
melodic_minor = [1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]
harmonic_minor = [1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1]
harmonic_major = [1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1]
wholetone = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]  # 2 transpositions, 1 mode
octatonic = [1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1]  # 3 transpositions, 2 modes
augmented = [1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0]  # 4 transpositions, 2 modes

In [ ]:
scales = {
    "major": Necklace(major),
    "melodic_minor": Necklace(melodic_minor),
    "harmonic_major": Necklace(harmonic_major),
    "harmonic_minor": Necklace(harmonic_minor),
    "wholetone": Necklace(wholetone),
    "octatonic": Necklace(octatonic),
    "augmented": Necklace(augmented),
}

In [ ]:
def get_children(scale):
    index = -1
    children = []
    for idx in range(sum(scale)):
        index = scale.index(1, index + 1)
        current = copy(scale)
        current[index] = 0
        # TODO: return tuple of child and number of occurrences
        if current not in children:
            children.append(current)

    return children

In [ ]:
chords = defaultdict(list)

chords[8].append(scales["octatonic"])
chords[7].extend(
    [scales["major"], scales["melodic_minor"], scales["harmonic_minor"], scales["harmonic_major"]]
)
chords[6].extend([scales["augmented"], scales["wholetone"]])

In [ ]:
for n in range(8, 1, -1):
    for chord in chords[n]:
        for sub_chord in get_children(chord):
            if sub_chord not in chords[n - 1]:
                chords[n - 1].append(sub_chord)

In [ ]:
for key, value in chords.items():
    print(f"number of {key} voice chords = {len(value)}")